# Srock Strategy - Buy & Hold 0001 - Selected Stock Buy & Hold

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2015-12-31'
ed = '2026-02-13'

In [3]:
instrument_list = ['000001.SZ', '000002.SZ', '600519.SH']

In [4]:
sql = f"""
WITH
data_buy AS (
    WITH
    data_min_date AS (
        SELECT
            date,
            instrument,
            MIN(date) OVER (PARTITION BY instrument) AS _min_date,
        FROM cn_stock_bar1d
        WHERE instrument IN {str(tuple(instrument_list))}
        QUALIFY (date = _min_date)
        ORDER by date, instrument
    )
    SELECT
        date,
        instrument,
        1 / SUM(1) OVER (PARTITION BY 1) AS position
    FROM data_min_date
),
data_sell AS (
    SELECT
        date,
        instrument,
        MAX(date) OVER (PARTITION BY instrument) AS _max_date,
        0 AS position,
    FROM cn_stock_bar1d
    WHERE instrument IN {str(tuple(instrument_list))}
    QUALIFY (date = _max_date)
    ORDER by date, instrument
)
SELECT * 
FROM data_buy 
UNION
SELECT * 
FROM data_sell
ORDER BY date, instrument
"""

In [5]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,position
0,2015-12-31,000001.SZ,0.333333
1,2015-12-31,000002.SZ,0.333333
2,2015-12-31,600519.SH,0.333333
3,2026-02-13,000001.SZ,0.000000
4,2026-02-13,000002.SZ,0.000000
5,2026-02-13,600519.SH,0.000000


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 14:06:11] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 14:06:11] [info     ] 2015-12-31, 2026-02-13, , , instruments=3
[2026-02-18 14:06:11] [info     ] bigtrader module V2.2.0
[2026-02-18 14:06:11] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
order_target_percent: price is null for '000002.SZ' at '2015-12-31 15:00:00'
[2026-02-18 14:06:14] [info     ] backtest done, raw_perf_ds:dai.DataSource("_817c49a4e9f84c8a9e7c5e25a5f8e991")


[2026-02-18 14:06:16] [info     ] bigtrader.v34 运行完成 [4.726s].
